# Object Detection using TAO YOLOv4

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://d29g4g2dyqv443.cloudfront.net/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png" width="1080">


## Sample prediction of YOLOv4
<img align="center" src="https://github.com/vpraveen-nv/model_card_images/blob/main/cv/notebook/common/sample.jpg?raw=true" width="960"> 

## Learning Objectives
In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Take a pretrained resnet18 model and train a ResNet-18 Yolo_v4 model on the KITTI dataset
* Prune the trained yolo_V4 model
* Retrain the pruned model to recover lost accuracy
* Export the pruned model
* Quantize the pruned model using QAT
* Run Inference on the trained model
* Export the pruned, quantized and retrained model to a .onnx file for deployment to DeepStream
* Run inference on the exported .onnx model to verify deployment using TensorRT

At the end of this notebook, you will have generated a trained and optimized `YOLOv4` model
which you may deploy via [Triton](https://github.com/NVIDIA-AI-IOT/tao-toolkit-triton-apps)
or [DeepStream](https://developer.nvidia.com/deepstream-sdk).

## Table of Contents

This notebook shows an example usecase of YOLO v4 object detection using Train Adapt Optimize (TAO) Toolkit.

0. [Set up env variables and map drives](#head-0)
1. [Install the TAO launcher](#head-1)
2. [Prepare dataset and pre-trained model](#head-2) <br>
     2.1 [Download the dataset](#head-2-1)<br>
     2.2 [Verify the downloaded dataset](#head-2-2)<br>
     2.3 [Generate tfrecords](#head-2-3)<br>
     2.4 [Download pretrained model](#head-2-4)
3. [Provide training specification](#head-3)
4. [Run TAO training](#head-4)
5. [Evaluate trained models](#head-5)
6. [Prune trained models](#head-6)
7. [Retrain pruned models](#head-7)
8. [Evaluate retrained model](#head-8)
9. [Visualize inferences](#head-9)
10. [Model Export](#head-10)
11. [Verify deployed model](#head-11)
12. [QAT workflow](#head-12) <br>
    12.1 [QAT Training](#head-12-1) <br>
    12.2. [QAT Evaluation](#head-12-2) <br>
    12.3. [Pruning QAT model](#head-12-3)<br>
    12.4. [Retraining](#head-12-4)<br>
    12.5. [Evaluation of the retrained model](#head-12-5)<br>
    12.6. [Inference of the retrained QAT model](#head-12-6)<br>
    12.7. [Deployment of the QAT model](#head-12-7)<br>
    12.8. [Verify the deployed QAT model](#head-12-8)<br>


## 0. Set up env variables and map drives <a class="anchor" id="head-0"></a>

The following notebook requires the user to set an env variable called the `$LOCAL_PROJECT_DIR` as the path to the users workspace. Please note that the dataset to run this notebook is expected to reside in the `$LOCAL_PROJECT_DIR/data`, while the TAO experiment generated collaterals will be output to `$LOCAL_PROJECT_DIR/yolo_v4`. More information on how to set up the dataset and the supported steps in the TAO workflow are provided in the subsequent cells.

*Note: Please make sure to remove any stray artifacts/files from the `$USER_EXPERIMENT_DIR` or `$DATA_DOWNLOAD_DIR` paths as mentioned below, that may have been generated from previous experiments. Having checkpoint files etc may interfere with creating a training graph for a new experiment.*

In [1]:
# Setting up env variables for cleaner command line commands.
import os

%env USER_EXPERIMENT_DIR=/workspace/tao-experiments/yolo_v4
%env DATA_DOWNLOAD_DIR=/workspace/tao-experiments/data

# Set this path if you don't run the notebook from the samples directory.
# %env NOTEBOOK_ROOT=~/tao-samples/yolo_v4

# Please define this local project directory that needs to be mapped to the TAO docker session.
# The dataset expected to be present in $LOCAL_PROJECT_DIR/data, while the results for the steps
# in this notebook will be stored at $LOCAL_PROJECT_DIR/yolo_v4
%env LOCAL_PROJECT_DIR=YOUR_LOCAL_PROJECT_DIR_PATH
os.environ["LOCAL_DATA_DIR"] = os.path.join(os.getenv("LOCAL_PROJECT_DIR", os.getcwd()), "data")
os.environ["LOCAL_EXPERIMENT_DIR"] = os.path.join(os.getenv("LOCAL_PROJECT_DIR", os.getcwd()), "yolo_v4")

# The sample spec files are present in the same path as the downloaded samples.
os.environ["LOCAL_SPECS_DIR"] = os.path.join(
    os.getenv("NOTEBOOK_ROOT", os.getcwd()),
    "specs"
)
%env SPECS_DIR=/workspace/tao-experiments/yolo_v4/specs

# Showing list of specification files.
!ls -rlt $LOCAL_SPECS_DIR

env: USER_EXPERIMENT_DIR=/workspace/tao-experiments/yolo_v4
env: DATA_DOWNLOAD_DIR=/workspace/tao-experiments/data
env: LOCAL_PROJECT_DIR=YOUR_LOCAL_PROJECT_DIR_PATH
env: SPECS_DIR=/workspace/tao-experiments/yolo_v4/specs
total 32
-rwxrwxrwx 1 root root 2488 Nov 30 01:49 yolo_v4_retrain_resnet18_kitti.txt
-rwxrwxrwx 1 root root 2477 Nov 30 01:49 yolo_v4_retrain_resnet18_kitti_qat.txt
-rwxrwxrwx 1 root root 2600 Nov 30 01:49 yolo_v4_retrain_resnet18_kitti_16bit_grayscale.txt
-rwxrwxrwx 1 root root  330 Nov 30 01:49 yolo_v4_tfrecords_kitti_train.txt
-rwxrwxrwx 1 root root 2457 Nov 30 01:49 yolo_v4_retrain_resnet18_kitti_seq.txt
-rwxrwxrwx 1 root root  320 Nov 30 01:49 yolo_v4_tfrecords_kitti_val.txt
-rwxrwxrwx 1 root root  346 Nov 30 01:49 yolo_v4_tfrecords_kitti_train_16bit_grayscale.txt
-rwxrwxrwx 1 root root  336 Nov 30 01:49 yolo_v4_tfrecords_kitti_val_16bit_grayscale.txt
-rwxrwxrwx 1 root root 2509 Nov 30 01:49 yolo_v4_train_resnet18_kitti.txt
-rwxrwxrwx 1 root root 2490 Nov 30 01:4

In [2]:
# Create local dir
!mkdir -p $LOCAL_DATA_DIR
!mkdir -p $LOCAL_EXPERIMENT_DIR

The cell below maps the project directory on your local host to a workspace directory in the TAO docker instance, so that the data and the results are mapped from outside to inside of the docker instance.

In [4]:
# Mapping up the local directories to the TAO docker.
import json
mounts_file = os.path.expanduser("~/.tao_mounts.json")

# Define the dictionary with the mapped drives
drive_map = {
    "Mounts": [
        # Mapping the data directory
        {
            "source": os.environ["LOCAL_PROJECT_DIR"],
            "destination": "/workspace/tao-experiments"
        },
        # Mapping the specs directory.
        {
            "source": os.environ["LOCAL_SPECS_DIR"],
            "destination": os.environ["SPECS_DIR"]
        },
    ]
}

# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(drive_map, mfile, indent=4)

In [5]:
!cat ~/.tao_mounts.json

{
    "Mounts": [
        {
            "source": "YOUR_LOCAL_PROJECT_DIR_PATH",
            "destination": "/workspace/tao-experiments"
        },
        {
            "source": "/workspaces/tao_tutorials/notebooks/tao_launcher_starter_kit/yolo_v4/specs",
            "destination": "/workspace/tao-experiments/yolo_v4/specs"
        }
    ]
}

## 1. Install the TAO launcher <a class="anchor" id="head-1"></a>
The TAO launcher is a python package distributed as a python wheel listed in PyPI. You may install the launcher by executing the following cell.

Please note that TAO Toolkit recommends users to run the TAO launcher in a virtual env with python 3.6.9. You may follow the instruction in this [page](https://virtualenvwrapper.readthedocs.io/en/latest/install.html) to set up a python virtual env using the `virtualenv` and `virtualenvwrapper` packages. Once you have setup virtualenvwrapper, please set the version of python to be used in the virtual env by using the `VIRTUALENVWRAPPER_PYTHON` variable. You may do so by running

```sh
export VIRTUALENVWRAPPER_PYTHON=/path/to/bin/python3.x
```
where x >= 6 and <= 8

We recommend performing this step first and then launching the notebook from the virtual environment. In addition to installing TAO python package, please make sure of the following software requirements:
* python >=3.7, <=3.10.x
* docker-ce > 19.03.5
* docker-API 1.40
* nvidia-container-toolkit > 1.3.0-1
* nvidia-container-runtime > 3.4.0-1
* nvidia-docker2 > 2.5.0-1
* nvidia-driver > 455+

Once you have installed the pre-requisites, please log in to the docker registry nvcr.io by following the command below

```sh
docker login nvcr.io
```

You will be triggered to enter a username and password. The username is `$oauthtoken` and the password is the API key generated from `ngc.nvidia.com`. Please follow the instructions in the [NGC setup guide](https://docs.nvidia.com/ngc/ngc-overview/index.html#generating-api-key) to generate your own API key.

After setting up your virtual environment with the above requirements, install TAO pip package.

In [6]:
# SKIP this step IF you have already installed the TAO launcher.
!pip3 install nvidia-tao

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [7]:
# View the versions of the TAO launcher
!tao info

Configuration of the TAO Toolkit Instance
task_group: ['model', 'dataset', 'deploy']
format_version: 3.0
toolkit_version: 5.5.0
published_date: 08/26/2024


## 2. Prepare dataset and pre-trained model <a class="anchor" id="head-2"></a>

 We will be using the KITTI detection dataset for the tutorial. To find more details please visit
 http://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=2d. Please download the KITTI detection images (http://www.cvlibs.net/download.php?file=data_object_image_2.zip) and labels (http://www.cvlibs.net/download.php?file=data_object_label_2.zip) to $DATA_DOWNLOAD_DIR.
 
 The data will then be extracted to have
 * training images in `$LOCAL_DATA_DIR/training/image_2`
 * training labels in `$LOCAL_DATA_DIR/training/label_2`
 * testing images in `$LOCAL_DATA_DIR/testing/image_2`
 
You may use this notebook with your own dataset as well. To use this example with your own dataset, please follow the same directory structure as mentioned below.

*Note: There are no labels for the testing images, therefore we use it just to visualize inferences for the trained model.*

### 2.1 Download the dataset <a class="anchor" id="head-2-1"></a>

Once you have gotten the download links in your email, please populate them in place of the `KITTI_IMAGES_DOWNLOAD_URL` and the `KITTI_LABELS_DOWNLOAD_URL`. This next cell, will download the data and place in `$LOCAL_DATA_DIR`

KITTI_IMAGES_DOWNLOAD_URL

https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_image_2.zip

KITTI_LABELS_DOWNLOAD_URL

https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_label_2.zip

In [9]:
import os
!mkdir -p $LOCAL_DATA_DIR
# os.environ["URL_IMAGES"]=KITTI_IMAGES_DOWNLOAD_URL
os.environ["URL_IMAGES"]= "https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_image_2.zip"
!if [ ! -f $LOCAL_DATA_DIR/data_object_image_2.zip ]; then wget $URL_IMAGES -O $LOCAL_DATA_DIR/data_object_image_2.zip; else echo "image archive already downloaded"; fi 
# os.environ["URL_LABELS"]=KITTI_LABELS_DOWNLOAD_URL
os.environ["URL_LABELS"]="https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_label_2.zip"
!if [ ! -f $LOCAL_DATA_DIR/data_object_label_2.zip ]; then wget $URL_LABELS -O $LOCAL_DATA_DIR/data_object_label_2.zip; else echo "label archive already downloaded"; fi 

--2024-12-03 22:00:58--  https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_image_2.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 3.5.136.85, 52.219.170.125, 3.5.138.151, ...
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|3.5.136.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12569945557 (12G) [application/zip]
Saving to: ‘YOUR_LOCAL_PROJECT_DIR_PATH/data/data_object_image_2.zip’

YOUR_LOCAL_PROJECT_ 100%[===================>]  11.71G  19.0MB/s    in 19m 7s  

2024-12-03 22:19:48 (10.4 MB/s) - ‘YOUR_LOCAL_PROJECT_DIR_PATH/data/data_object_image_2.zip’ saved [12569945557/12569945557]

--2024-12-03 22:19:48--  https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_label_2.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.140.75, 52.219.170.133, 3.5.135.184, ...
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.140.7

### 2.2 Verify the downloaded dataset <a class="anchor" id="head-2-2"></a>

In [10]:
# Check the dataset is present
!mkdir -p $LOCAL_DATA_DIR
!if [ ! -f $LOCAL_DATA_DIR/data_object_image_2.zip ]; then echo 'Image zip file not found, please download.'; else echo 'Found Image zip file.';fi
!if [ ! -f $LOCAL_DATA_DIR/data_object_label_2.zip ]; then echo 'Label zip file not found, please download.'; else echo 'Found Labels zip file.';fi

Found Image zip file.
Found Labels zip file.


In [11]:
# This may take a while: verify integrity of zip files 
!sha256sum $LOCAL_DATA_DIR/data_object_image_2.zip | cut -d ' ' -f 1 | grep -xq '^351c5a2aa0cd9238b50174a3a62b846bc5855da256b82a196431d60ff8d43617$' ; \
if test $? -eq 0; then echo "images OK"; else echo "images corrupt, re-download!" && rm -f $LOCAL_DATA_DIR/data_object_image_2.zip; fi 
!sha256sum $LOCAL_DATA_DIR/data_object_label_2.zip | cut -d ' ' -f 1 | grep -xq '^4efc76220d867e1c31bb980bbf8cbc02599f02a9cb4350effa98dbb04aaed880$' ; \
if test $? -eq 0; then echo "labels OK"; else echo "labels corrupt, re-download!" && rm -f $LOCAL_DATA_DIR/data_object_label_2.zip; fi 

images OK
labels OK


In [12]:
# unpack 
!unzip -u $LOCAL_DATA_DIR/data_object_image_2.zip -d $LOCAL_DATA_DIR
!unzip -u $LOCAL_DATA_DIR/data_object_label_2.zip -d $LOCAL_DATA_DIR

Archive:  YOUR_LOCAL_PROJECT_DIR_PATH/data/data_object_image_2.zip
   creating: YOUR_LOCAL_PROJECT_DIR_PATH/data/training/image_2/
 extracting: YOUR_LOCAL_PROJECT_DIR_PATH/data/training/image_2/002480.png  
 extracting: YOUR_LOCAL_PROJECT_DIR_PATH/data/training/image_2/005952.png  
 extracting: YOUR_LOCAL_PROJECT_DIR_PATH/data/training/image_2/000709.png  
 extracting: YOUR_LOCAL_PROJECT_DIR_PATH/data/training/image_2/000814.png  
 extracting: YOUR_LOCAL_PROJECT_DIR_PATH/data/training/image_2/006192.png  
 extracting: YOUR_LOCAL_PROJECT_DIR_PATH/data/training/image_2/006017.png  
 extracting: YOUR_LOCAL_PROJECT_DIR_PATH/data/training/image_2/002731.png  
 extracting: YOUR_LOCAL_PROJECT_DIR_PATH/data/training/image_2/005295.png  
 extracting: YOUR_LOCAL_PROJECT_DIR_PATH/data/training/image_2/005347.png  
 extracting: YOUR_LOCAL_PROJECT_DIR_PATH/data/training/image_2/005326.png  
 extracting: YOUR_LOCAL_PROJECT_DIR_PATH/data/training/image_2/005713.png  
 extracting: YOUR_LOCAL_PROJECT_D

In [13]:
# verify
import os

DATA_DIR = os.environ.get('LOCAL_DATA_DIR')
num_training_images = len(os.listdir(os.path.join(DATA_DIR, "training/image_2")))
num_training_labels = len(os.listdir(os.path.join(DATA_DIR, "training/label_2")))
num_testing_images = len(os.listdir(os.path.join(DATA_DIR, "testing/image_2")))
print("Number of images in the train/val set. {}".format(num_training_images))
print("Number of labels in the train/val set. {}".format(num_training_labels))
print("Number of images in the test set. {}".format(num_testing_images))

Number of images in the train/val set. 7481
Number of labels in the train/val set. 7481
Number of images in the test set. 7518


In [14]:
# Directory where splitted dataset will be located
!mkdir -p $LOCAL_DATA_DIR/kitti_split
# Generate val dataset out of training dataset
!python3 ../ssd/generate_split.py --input_image_dir=$LOCAL_DATA_DIR/training/image_2 \
                                  --input_label_dir=$LOCAL_DATA_DIR/training/label_2 \
                                  --output_dir=$LOCAL_DATA_DIR/kitti_split

Total 7481 samples in KITTI training dataset
6733 for train and 748 for val
Finished copying training set
Finished copying validation set


Additionally, if you have your own dataset already in a volume (or folder), you can mount the volume on `LOCAL_DATA_DIR` (or create a soft link). Below shows an example:
```bash
# if your dataset is in /dev/sdc1
mount /dev/sdc1 $LOCAL_DATA_DIR

# if your dataset is in folder /var/dataset
ln -sf /var/dataset $LOCAL_DATA_DIR
```

In [ ]:
# If you use your own dataset, you will need to run the code below to generate the best anchor shape

# !tao model yolo_v4 kmeans -l $DATA_DOWNLOAD_DIR/kitti_split/training/label \
#                     -i $DATA_DOWNLOAD_DIR/kitti_split/training/image \
#                     -n 9 \
#                     -x 1248 \
#                     -y 384

# The anchor shape generated by this script is sorted. Write the first 3 into small_anchor_shape in the config
# file. Write middle 3 into mid_anchor_shape. Write last 3 into big_anchor_shape.

### 2.3 Generate tfrecords <a class="anchor" id="head-2-3"></a>

The default YOLOv4 data format requires generation of TFRecords. Currently, the old sequence data format (image folders and label txt folders) is still supported and if you prefer to use the sequence data format, you can skip this section. To use sequence data format, please use spec file `yolo_v4_train_resnet18_kitti_seq.txt` and `yolo_v4_retrain_resnet18_kitti_seq.txt`. And you can check our [user guide](https://docs.nvidia.com/tao/tao-toolkit/text/object_detection/yolo_v4.html#dataset-config) for more details about tfrecords generation and sequence data format usage.

Note: we observe that for YOLOv4, when mosaic augmentation is turned on (mosaic_prob > 0), the sequence format has faster training speed.

Note: we observe the TFRecords format sometimes results in CUDA error during evaluation. Setting `force_on_cpu` in `nms_config` to `true` can help prevent this problem.

In [15]:
!tao model yolo_v4 dataset_convert -d $SPECS_DIR/yolo_v4_tfrecords_kitti_train.txt \
                             -o $DATA_DOWNLOAD_DIR/yolo_v4/tfrecords/train \
                             -r $USER_EXPERIMENT_DIR/

Traceback (most recent call last):
  File "/home/vscode/.local/bin/tao", line 8, in <module>
    sys.exit(main())
  File "/home/vscode/.local/lib/python3.10/site-packages/nvidia_tao_cli/entrypoint/tao_launcher.py", line 134, in main
    instance.launch_command(
  File "/home/vscode/.local/lib/python3.10/site-packages/nvidia_tao_cli/components/instance_handler/local_instance.py", line 356, in launch_command
    docker_logged_in(required_registry=task_map[task].docker_registry)
  File "/home/vscode/.local/lib/python3.10/site-packages/nvidia_tao_cli/components/instance_handler/utils.py", line 151, in docker_logged_in
    data = load_config_file(docker_config)
  File "/home/vscode/.local/lib/python3.10/site-packages/nvidia_tao_cli/components/instance_handler/utils.py", line 84, in load_config_file
    assert os.path.exists(config_path), (
AssertionError: Config path must be a valid unix path. No file found at: /home/vscode/.docker/config.json. Did you run docker login?


In [16]:
!tao model yolo_v4 dataset_convert -d $SPECS_DIR/yolo_v4_tfrecords_kitti_val.txt \
                             -o $DATA_DOWNLOAD_DIR/yolo_v4/tfrecords/val \
                             -r $USER_EXPERIMENT_DIR/

Traceback (most recent call last):
  File "/home/vscode/.local/bin/tao", line 8, in <module>
    sys.exit(main())
  File "/home/vscode/.local/lib/python3.10/site-packages/nvidia_tao_cli/entrypoint/tao_launcher.py", line 134, in main
    instance.launch_command(
  File "/home/vscode/.local/lib/python3.10/site-packages/nvidia_tao_cli/components/instance_handler/local_instance.py", line 356, in launch_command
    docker_logged_in(required_registry=task_map[task].docker_registry)
  File "/home/vscode/.local/lib/python3.10/site-packages/nvidia_tao_cli/components/instance_handler/utils.py", line 151, in docker_logged_in
    data = load_config_file(docker_config)
  File "/home/vscode/.local/lib/python3.10/site-packages/nvidia_tao_cli/components/instance_handler/utils.py", line 84, in load_config_file
    assert os.path.exists(config_path), (
AssertionError: Config path must be a valid unix path. No file found at: /home/vscode/.docker/config.json. Did you run docker login?


### 2.4 Download pre-trained model <a class="anchor" id="head-2-4"></a>

We will use NGC CLI to get the pre-trained models. For more details, go to [ngc.nvidia.com](ngc.nvidia.com) and click the SETUP on the navigation bar.

In [17]:
# Installing NGC CLI on the local machine.
## Download and install
%env CLI=ngccli_cat_linux.zip
!mkdir -p $LOCAL_PROJECT_DIR/ngccli

# Remove any previously existing CLI installations
!rm -rf $LOCAL_PROJECT_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli
!unzip -u "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
!rm $LOCAL_PROJECT_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli/ngc-cli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""), os.getenv("PATH", ""))

env: CLI=ngccli_cat_linux.zip
--2024-12-03 23:13:17--  https://ngc.nvidia.com/downloads/ngccli_cat_linux.zip
Resolving ngc.nvidia.com (ngc.nvidia.com)... 108.156.152.11, 108.156.152.30, 108.156.152.17, ...
Connecting to ngc.nvidia.com (ngc.nvidia.com)|108.156.152.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47816771 (46M) [application/zip]
Saving to: ‘YOUR_LOCAL_PROJECT_DIR_PATH/ngccli/ngccli_cat_linux.zip’

ngccli_cat_linux.zi 100%[===================>]  45.60M  14.6MB/s    in 3.1s    

2024-12-03 23:13:21 (14.6 MB/s) - ‘YOUR_LOCAL_PROJECT_DIR_PATH/ngccli/ngccli_cat_linux.zip’ saved [47816771/47816771]

Archive:  YOUR_LOCAL_PROJECT_DIR_PATH/ngccli/ngccli_cat_linux.zip
   creating: YOUR_LOCAL_PROJECT_DIR_PATH/ngccli/ngc-cli/
   creating: YOUR_LOCAL_PROJECT_DIR_PATH/ngccli/ngc-cli/multidict/
  inflating: YOUR_LOCAL_PROJECT_DIR_PATH/ngccli/ngc-cli/multidict/_multidict.cpython-39-x86_64-linux-gnu.so  
   creating: YOUR_LOCAL_PROJECT_DIR_PATH/ngccli/ngc-cli/

In [18]:
!ngc registry model list nvidia/tao/pretrained_object_detection:*

+-------+-------+-------+-------+-------+-------+-------+-------+-------+
| Versi | Accur | Epoch | Batch | GPU   | Memor | File  | Statu | Creat |
| on    | acy   | s     | Size  | Model | y Foo | Size  | s     | ed    |
|       |       |       |       |       | tprin |       |       | Date  |
|       |       |       |       |       | t     |       |       |       |
+-------+-------+-------+-------+-------+-------+-------+-------+-------+
| vgg19 | 77.56 | 80    | 1     | V100  | 153.7 | 153.7 | UPLOA | Aug   |
|       |       |       |       |       |       | 2 MB  | D_COM | 18,   |
|       |       |       |       |       |       |       | PLETE | 2021  |
| vgg16 | 77.17 | 80    | 1     | V100  | 113.2 | 113.1 | UPLOA | Aug   |
|       |       |       |       |       |       | 6 MB  | D_COM | 18,   |
|       |       |       |       |       |       |       | PLETE | 2021  |
| squee | 65.13 | 80    | 1     | V100  | 6.5   | 6.46  | UPLOA | Aug   |
| zenet |       |       |       |     

In [19]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/pretrained_resnet18/

In [20]:
# Pull pretrained model from NGC
!ngc registry model download-version nvidia/tao/pretrained_object_detection:resnet18 \
                    --dest $LOCAL_EXPERIMENT_DIR/pretrained_resnet18

Getting files to download...
⠋ ━━ • 0… • Remaining: … • ? • Elapsed: 0… • Total: 1 - Completed: 0 - Failed: 0
⠙ ━━ • 0… • Remaining: … • ? • Elapsed: 0… • Total: 1 - Completed: 0 - Failed: 0
⠹ ━━ • 0… • Remaining: … • ? • Elapsed: 0… • Total: 1 - Completed: 0 - Failed: 0
⠼ ━━ • 0… • Remaining: … • ? • Elapsed: 0… • Total: 1 - Completed: 0 - Failed: 0
⠴ ━━ • 0… • Remaining: … • ? • Elapsed: 0… • Total: 1 - Completed: 0 - Failed: 0
⠦ ━━ • 0… • Remaining: … • ? • Elapsed: 0… • Total: 1 - Completed: 0 - Failed: 0
⠇ ━━ • 0… • Remaining: … • ? • Elapsed: 0… • Total: 1 - Completed: 0 - Failed: 0
⠏ ━━ • 0… • Remaining: … • ? • Elapsed: 0… • Total: 1 - Completed: 0 - Failed: 0
⠙ ━━ • 0… • Remaining: … • ? • Elapsed: 0… • Total: 1 - Completed: 0 - Failed: 0
⠹ ━━ • 0… • Remaining: … • ? • Elapsed: 0… • Total: 1 - Completed: 0 - Failed: 0
⠸ ━━ • 0… • Remaining: … • ? • Elapsed: 0… • Total: 1 - Completed: 0 - Failed: 0
⠴ ━━ • 0… • Remaining: … • ? • Elapsed: 0… • Total: 1 - Completed: 0 - Failed: 0

In [21]:
print("Check that model is downloaded into dir.")
!ls -l $LOCAL_EXPERIMENT_DIR/pretrained_resnet18/pretrained_object_detection_vresnet18

Check that model is downloaded into dir.
total 91096
-rw-r--r-- 1 vscode vscode 93278448 Dec  3 23:21 resnet_18.hdf5


## 3. Provide training specification <a class="anchor" id="head-3"></a>
* Augmentation parameters for on-the-fly data augmentation
* Other training (hyper-)parameters such as batch size, number of epochs, learning rate etc.
* Whether to use quantization aware training (QAT)

In [22]:
# Provide pretrained model path
!sed -i 's,EXPERIMENT_DIR,'"$USER_EXPERIMENT_DIR"',' $LOCAL_SPECS_DIR/yolo_v4_train_resnet18_kitti.txt

In [23]:
!cat $LOCAL_SPECS_DIR/yolo_v4_train_resnet18_kitti.txt

random_seed: 42
yolov4_config {
  big_anchor_shape: "[(114.94, 60.67), (159.06, 114.59), (297.59, 176.38)]"
  mid_anchor_shape: "[(42.99, 31.91), (79.57, 31.75), (56.80, 56.93)]"
  small_anchor_shape: "[(15.60, 13.88), (30.25, 20.25), (20.67, 49.63)]"
  box_matching_iou: 0.25
  matching_neutral_box_iou: 0.5
  arch: "resnet"
  nlayers: 18
  arch_conv_blocks: 2
  loss_loc_weight: 1.0
  loss_neg_obj_weights: 1.0
  loss_class_weights: 1.0
  label_smoothing: 0.0
  big_grid_xy_extend: 0.05
  mid_grid_xy_extend: 0.1
  small_grid_xy_extend: 0.2
  freeze_bn: false
  #freeze_blocks: 0
  force_relu: false
}
training_config {
  visualizer {
      enabled: False
      num_images: 3
  }
  batch_size_per_gpu: 8
  num_epochs: 80
  enable_qat: false
  checkpoint_interval: 10
  learning_rate {
    soft_start_cosine_annealing_schedule {
      min_learning_rate: 1e-7
      max_learning_rate: 1e-4
      soft_start: 0.3
    }
  }
  regularizer {
    type: L1
    weight: 3e-5
  }
  optimizer {
    adam {
   

## 4. Run TAO training <a class="anchor" id="head-4"></a>
* Provide the sample spec file and the output directory location for models
* WARNING: training will take several hours or one day to complete

In [24]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/experiment_dir_unpruned

In [25]:
print("To run with multigpu, please change --gpus based on the number of available GPUs in your machine.")
!tao model yolo_v4 train -e $SPECS_DIR/yolo_v4_train_resnet18_kitti.txt \
                   -r $USER_EXPERIMENT_DIR/experiment_dir_unpruned \
                   --gpus 1

To run with multigpu, please change --gpus based on the number of available GPUs in your machine.
Traceback (most recent call last):
  File "/home/vscode/.local/bin/tao", line 8, in <module>
    sys.exit(main())
  File "/home/vscode/.local/lib/python3.10/site-packages/nvidia_tao_cli/entrypoint/tao_launcher.py", line 134, in main
    instance.launch_command(
  File "/home/vscode/.local/lib/python3.10/site-packages/nvidia_tao_cli/components/instance_handler/local_instance.py", line 356, in launch_command
    docker_logged_in(required_registry=task_map[task].docker_registry)
  File "/home/vscode/.local/lib/python3.10/site-packages/nvidia_tao_cli/components/instance_handler/utils.py", line 151, in docker_logged_in
    data = load_config_file(docker_config)
  File "/home/vscode/.local/lib/python3.10/site-packages/nvidia_tao_cli/components/instance_handler/utils.py", line 84, in load_config_file
    assert os.path.exists(config_path), (
AssertionError: Config path must be a valid unix path. 

In [26]:
print("To resume from checkpoint, please change pretrain_model_path to resume_model_path in config file.")

To resume from checkpoint, please change pretrain_model_path to resume_model_path in config file.


In [27]:
print('Model for each epoch:')
print('---------------------')
!ls -ltrh $LOCAL_EXPERIMENT_DIR/experiment_dir_unpruned/weights

Model for each epoch:
---------------------
ls: cannot access 'YOUR_LOCAL_PROJECT_DIR_PATH/yolo_v4/experiment_dir_unpruned/weights': No such file or directory


In [ ]:
# Now check the evaluation stats in the csv file and pick the model with highest eval accuracy.
!cat $LOCAL_EXPERIMENT_DIR/experiment_dir_unpruned/yolov4_training_log_resnet18.csv
%set_env EPOCH=080

## 5. Evaluate trained models <a class="anchor" id="head-5"></a>

In [ ]:
!tao model yolo_v4 evaluate -e $SPECS_DIR/yolo_v4_train_resnet18_kitti.txt \
                      -m $USER_EXPERIMENT_DIR/experiment_dir_unpruned/weights/yolov4_resnet18_epoch_$EPOCH.hdf5

## 6. Prune trained models <a class="anchor" id="head-6"></a>
* Specify pre-trained model
* Equalization criterion (`Only for resnets as they have element wise operations or MobileNets.`)
* Threshold for pruning.
* Output directory to store the model

Usually, you just need to adjust `-pth` (threshold) for accuracy and model size trade off. Higher `pth` gives you smaller model (and thus higher inference speed) but worse accuracy. The threshold value depends on the dataset and the model. `0.5` in the block below is just a start point. If the retrain accuracy is good, you can increase this value to get smaller models. Otherwise, lower this value to get better accuracy.

In [ ]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/experiment_dir_pruned

In [ ]:
!tao model yolo_v4 prune -m $USER_EXPERIMENT_DIR/experiment_dir_unpruned/weights/yolov4_resnet18_epoch_$EPOCH.hdf5 \
                   -e $SPECS_DIR/yolo_v4_train_resnet18_kitti.txt \
                   -o $USER_EXPERIMENT_DIR/experiment_dir_pruned/yolov4_resnet18_pruned.hdf5 \
                   -eq intersection \
                   -pth 0.1

In [ ]:
!ls -rlt $LOCAL_EXPERIMENT_DIR/experiment_dir_pruned/

## 7. Retrain pruned models <a class="anchor" id="head-7"></a>
* Model needs to be re-trained to bring back accuracy after pruning
* Specify re-training specification
* WARNING: training will take several hours or one day to complete

In [ ]:
# Printing the retrain spec file. 
# Here we have updated the spec file to include the newly pruned model as a pretrained weights.
!sed -i 's,EXPERIMENT_DIR,'"$USER_EXPERIMENT_DIR"',' $LOCAL_SPECS_DIR/yolo_v4_retrain_resnet18_kitti.txt
!cat $LOCAL_SPECS_DIR/yolo_v4_retrain_resnet18_kitti.txt

In [ ]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/experiment_dir_retrain

In [ ]:
# Retraining using the pruned model as pretrained weights 
!tao model yolo_v4 train --gpus 1 \
                   -e $SPECS_DIR/yolo_v4_retrain_resnet18_kitti.txt \
                   -r $USER_EXPERIMENT_DIR/experiment_dir_retrain

In [ ]:
# Listing the newly retrained model.
!ls -rlt $LOCAL_EXPERIMENT_DIR/experiment_dir_retrain/weights

In [ ]:
# Now check the evaluation stats in the csv file and pick the model with highest eval accuracy.
!cat $LOCAL_EXPERIMENT_DIR/experiment_dir_retrain/yolov4_training_log_resnet18.csv
%set_env EPOCH=080

## 8. Evaluate retrained model <a class="anchor" id="head-8"></a>

In [ ]:
!tao model yolo_v4 evaluate -e $SPECS_DIR/yolo_v4_retrain_resnet18_kitti.txt \
                      -m $USER_EXPERIMENT_DIR/experiment_dir_retrain/weights/yolov4_resnet18_epoch_$EPOCH.hdf5

## 9. Visualize inferences <a class="anchor" id="head-9"></a>
In this section, we run the `infer` tool to generate inferences on the trained models and visualize the results.

In [ ]:
# Copy some test images
!mkdir -p $LOCAL_DATA_DIR/test_samples
!cp $LOCAL_DATA_DIR/testing/image_2/00000* $LOCAL_DATA_DIR/test_samples/

In [ ]:
# Running inference for detection on n images
!tao model yolo_v4 inference -i $DATA_DOWNLOAD_DIR/test_samples \
                       -e $SPECS_DIR/yolo_v4_retrain_resnet18_kitti.txt \
                       -m $USER_EXPERIMENT_DIR/experiment_dir_retrain/weights/yolov4_resnet18_epoch_$EPOCH.hdf5 \
                       -r $USER_EXPERIMENT_DIR/

The `inference` tool produces two outputs. 
1. Overlain images in `$LOCAL_EXPERIMENT_DIR/images_annotated`
2. Frame by frame bbox labels in kitti format located in `$LOCAL_EXPERIMENT_DIR/labels`

In [ ]:
# Simple grid visualizer
!pip3 install "matplotlib>=3.3.3, <4.0"
import matplotlib.pyplot as plt
import os
from math import ceil
valid_image_ext = ['.jpg', '.png', '.jpeg', '.ppm']

def visualize_images(image_dir, num_cols=4, num_images=10):
    output_path = os.path.join(os.environ['LOCAL_EXPERIMENT_DIR'], image_dir)
    num_rows = int(ceil(float(num_images) / float(num_cols)))
    f, axarr = plt.subplots(num_rows, num_cols, figsize=[80,30])
    f.tight_layout()
    a = [os.path.join(output_path, image) for image in os.listdir(output_path) 
         if os.path.splitext(image)[1].lower() in valid_image_ext]
    for idx, img_path in enumerate(a[:num_images]):
        col_id = idx % num_cols
        row_id = idx // num_cols
        img = plt.imread(img_path)
        axarr[row_id, col_id].imshow(img) 

In [ ]:
# Visualizing the sample images.
OUTPUT_PATH = 'images_annotated' # relative path from $USER_EXPERIMENT_DIR.
COLS = 3 # number of columns in the visualizer grid.
IMAGES = 9 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

## 10. Model Export <a class="anchor" id="head-10"></a>

If you trained a non-QAT model, you may export in FP32, FP16 or INT8 mode using the code block below. For INT8, you need to provide calibration image directory.

In [ ]:
# tao <task> export will fail if .onnx already exists. So we clear the export folder before tao <task> export
!rm -rf $LOCAL_EXPERIMENT_DIR/export
!mkdir -p $LOCAL_EXPERIMENT_DIR/export
# Generate .onnx file using tao container
!tao model yolo_v4 export -m $USER_EXPERIMENT_DIR/experiment_dir_retrain/weights/yolov4_resnet18_epoch_$EPOCH.hdf5 \
                    -o $USER_EXPERIMENT_DIR/export/yolov4_resnet18_epoch_$EPOCH.onnx \
                    -e $SPECS_DIR/yolo_v4_retrain_resnet18_kitti.txt \
                    --target_opset 12 \
                    --gen_ds_config

Using the `tao deploy` container, you can generate a TensorRT engine and verify the correctness of the generated through evaluate and inference.

The `tao deploy` produces optimized tensorrt engines for the platform that it resides on. Therefore, to get maximum performance, please run `tao deploy` command which will instantiate a deploy container, with the exported `.onnx` file on your target device. The `tao deploy` container only works for x86, with discrete NVIDIA GPU's.

For the jetson devices, please download the tao-converter for jetson and refer to [here](https://docs.nvidia.com/tao/tao-toolkit-archive/tao-30-2108/text/tensorrt.html#installing-the-tao-converter) for more details.

If you choose to integrate your model into deepstream directly, you may do so by simply copying the exported `.onnx` file along with the calibration cache to the target device and updating the spec file that configures the `gst-nvinfer` element to point to this newly exported model. Usually this file is called `config_infer_primary.txt` for detection models and `config_infer_secondary_*.txt` for classification models.

In [ ]:
# Convert to TensorRT engine (FP32). 
!tao deploy yolo_v4 gen_trt_engine -m $USER_EXPERIMENT_DIR/export/yolov4_resnet18_epoch_$EPOCH.onnx \
                                   -e $SPECS_DIR/yolo_v4_retrain_resnet18_kitti.txt \
                                   --batch_size 16 \
                                   --min_batch_size 1 \
                                   --opt_batch_size 8 \
                                   --max_batch_size 16 \
                                   --data_type fp32 \
                                   --engine_file $USER_EXPERIMENT_DIR/export/trt.engine \
                                   --results_dir $USER_EXPERIMENT_DIR/export

In [ ]:
# Convert to TensorRT engine (FP16). 
!tao deploy yolo_v4 gen_trt_engine -m $USER_EXPERIMENT_DIR/export/yolov4_resnet18_epoch_$EPOCH.onnx \
                                   -e $SPECS_DIR/yolo_v4_retrain_resnet18_kitti.txt \
                                   --batch_size 16 \
                                   --min_batch_size 1 \
                                   --opt_batch_size 8 \
                                   --max_batch_size 16 \
                                   --data_type fp16 \
                                   --engine_file $USER_EXPERIMENT_DIR/export/trt.engine.fp16 \
                                   --results_dir $USER_EXPERIMENT_DIR/export

`Note:` In this example, for ease of execution we restrict the number of calibrating batches to 10. TAO Toolkit recommends the use of at least 10% of the training dataset for int8 calibration.

In [ ]:
# To export in INT8 mode (generate calibration cache file). 
!tao deploy yolo_v4 gen_trt_engine -m $USER_EXPERIMENT_DIR/export/yolov4_resnet18_epoch_$EPOCH.onnx \
                                   -e $SPECS_DIR/yolo_v4_retrain_resnet18_kitti.txt \
                                   --cal_image_dir $DATA_DOWNLOAD_DIR/testing/image_2 \
                                   --data_type int8 \
                                   --batch_size 16 \
                                   --min_batch_size 1 \
                                   --opt_batch_size 8 \
                                   --max_batch_size 16 \
                                   --batches 10 \
                                   --cal_cache_file $USER_EXPERIMENT_DIR/export/cal.bin  \
                                   --cal_data_file $USER_EXPERIMENT_DIR/export/cal.tensorfile \
                                   --engine_file $USER_EXPERIMENT_DIR/export/trt.engine.int8 \
                                   --results_dir $USER_EXPERIMENT_DIR/export

In [ ]:
print('Exported model:')
print('------------')
!ls -lh $LOCAL_EXPERIMENT_DIR/export

## 11. Verify the deployed model <a class="anchor" id="head-11"></a>
Verify the converted engine by visualizing TensorRT inferences.


In [ ]:
# Infer using TensorRT engine
!tao deploy yolo_v4 inference -m $USER_EXPERIMENT_DIR/export/trt.engine \
                              -e $SPECS_DIR/yolo_v4_retrain_resnet18_kitti.txt \
                              -i $DATA_DOWNLOAD_DIR/test_samples \
                              -r $USER_EXPERIMENT_DIR/yolo_infer_images \
                              -t 0.6

In [ ]:
# Visualizing the sample images.
OUTPUT_PATH = 'yolo_infer_images/images_annotated' # relative path from $USER_EXPERIMENT_DIR.
COLS = 3 # number of columns in the visualizer grid.
IMAGES = 9 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

## 12. QAT workflow <a class="anchor" id="head-12"></a>
In this section, we will explore the typical Quantization-Aware Training(QAT) workflow with TAO. QAT workflow is almost the same as non-QAT workflow except for two major differences:
1. set `enable_qat` to `True` in training and retraining spec files to enable the QAT for training/retraining
2. when doing export in INT8 mode, the calibration json file that stores the scales used during QAT is extracted during `tao <task> export`. The .onnx file and calibration json file are used to generate engine file through tao deploy

 ### 12.1. QAT Training <a class="anchor" id="head-12-1"></a>

In [ ]:
# To enable QAT training on sample spec file, we need to set `enable_qat` to `True` in training spec files
!sed -i "s/enable_qat: false/enable_qat: true/g" $LOCAL_SPECS_DIR/yolo_v4_train_resnet18_kitti_qat.txt
!sed -i 's,EXPERIMENT_DIR,'"$USER_EXPERIMENT_DIR"',' $LOCAL_SPECS_DIR/yolo_v4_train_resnet18_kitti_qat.txt
!cat $LOCAL_SPECS_DIR/yolo_v4_train_resnet18_kitti_qat.txt

In [ ]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/experiment_dir_unpruned_QAT

In [ ]:
print("To run with multigpu, please change --gpus based on the number of available GPUs in your machine.")
!tao model yolo_v4 train -e $SPECS_DIR/yolo_v4_train_resnet18_kitti_qat.txt \
                   -r $USER_EXPERIMENT_DIR/experiment_dir_unpruned_QAT \
                   --gpus 1

In [ ]:
print('Model for each epoch:')
print('---------------------')
!ls -ltrh $LOCAL_EXPERIMENT_DIR/experiment_dir_unpruned_QAT/weights

In [ ]:
# Now check the evaluation stats in the csv file and pick the model with highest eval accuracy.
!cat $LOCAL_EXPERIMENT_DIR/experiment_dir_unpruned_QAT/yolov4_training_log_resnet18.csv
%set_env EPOCH=080

 ### 12.2. QAT Evaluation <a class="anchor" id="head-12-2"></a>

In [ ]:
!tao model yolo_v4 evaluate -e $SPECS_DIR/yolo_v4_train_resnet18_kitti_qat.txt \
                      -m $USER_EXPERIMENT_DIR/experiment_dir_unpruned_QAT/weights/yolov4_resnet18_epoch_$EPOCH.hdf5

 ### 12.3. Pruning QAT model <a class="anchor" id="head-12-3"></a>

In [ ]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/experiment_dir_pruned_QAT

In [ ]:
!tao model yolo_v4 prune -m $USER_EXPERIMENT_DIR/experiment_dir_unpruned_QAT/weights/yolov4_resnet18_epoch_$EPOCH.hdf5 \
                   -e $SPECS_DIR/yolo_v4_train_resnet18_kitti_qat.txt \
                   -o $USER_EXPERIMENT_DIR/experiment_dir_pruned_QAT/yolov4_resnet18_qat_pruned.hdf5 \
                   -eq intersection \
                   -pth 0.1

In [ ]:
!ls -rlt $LOCAL_EXPERIMENT_DIR/experiment_dir_pruned_QAT/

 ### 12.4. Retraining <a class="anchor" id="head-12-4"></a>

In [ ]:
!sed -i "s/enable_qat: false/enable_qat: true/g" $LOCAL_SPECS_DIR/yolo_v4_retrain_resnet18_kitti_qat.txt
!sed -i 's,EXPERIMENT_DIR,'"$USER_EXPERIMENT_DIR"',' $LOCAL_SPECS_DIR/yolo_v4_retrain_resnet18_kitti_qat.txt
!cat $LOCAL_SPECS_DIR/yolo_v4_retrain_resnet18_kitti_qat.txt

In [ ]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/experiment_dir_retrain_qat

In [ ]:
# Retraining using the pruned model as pretrained weights 
!tao model yolo_v4 train --gpus 1 \
                   -e $SPECS_DIR/yolo_v4_retrain_resnet18_kitti_qat.txt \
                   -r $USER_EXPERIMENT_DIR/experiment_dir_retrain_qat

In [ ]:
# Listing the newly retrained model.
!ls -rlt $LOCAL_EXPERIMENT_DIR/experiment_dir_retrain_qat/weights

In [ ]:
# Now check the evaluation stats in the csv file and pick the model with highest eval accuracy.
!cat $LOCAL_EXPERIMENT_DIR/experiment_dir_retrain_qat/yolov4_training_log_resnet18.csv
%set_env EPOCH=080

 ### 12.5. Evaluation of the retrained model <a class="anchor" id="head-12-5"></a>

In [ ]:
!tao model yolo_v4 evaluate -e $SPECS_DIR/yolo_v4_retrain_resnet18_kitti_qat.txt \
                      -m $USER_EXPERIMENT_DIR/experiment_dir_retrain_qat/weights/yolov4_resnet18_epoch_$EPOCH.hdf5

 ### 12.6. Inference of the retrained QAT model <a class="anchor" id="head-12-6"></a>

In [ ]:
!tao model yolo_v4 inference -i $DATA_DOWNLOAD_DIR/test_samples \
                       -e $SPECS_DIR/yolo_v4_retrain_resnet18_kitti_qat.txt \
                       -m $USER_EXPERIMENT_DIR/experiment_dir_retrain_qat/weights/yolov4_resnet18_epoch_$EPOCH.hdf5 \
                       -r $USER_EXPERIMENT_DIR/

In [ ]:
# Visualizing the sample images.
OUTPUT_PATH = 'images_annotated' # relative path from $USER_EXPERIMENT_DIR.
COLS = 3 # number of columns in the visualizer grid.
IMAGES = 9 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)

 ### 12.7. Deployment of the QAT model <a class="anchor" id="head-12-7"></a>

 #### Generate .onnx file using tao container
If you train a QAT model, you may only export in INT8 mode using following code block. This generates an onnx file and the corresponding calibration json file that stores scales used during QAT. You can either use the onnx file and calibration json file to generate int8 engine through tao deploy or DeepStream for FP32 or FP16 mode. But please note this gives sub-optimal results. If you want to deploy in FP32 or FP16, you should disable QAT in training.

In [ ]:
!mkdir -p $LOCAL_EXPERIMENT_DIR/export_qat
!tao model yolo_v4 export -m $USER_EXPERIMENT_DIR/experiment_dir_retrain_qat/weights/yolov4_resnet18_epoch_$EPOCH.hdf5  \
                    -o $USER_EXPERIMENT_DIR/export_qat/yolov4_resnet18_epoch_$EPOCH.onnx \
                    -e $SPECS_DIR/yolo_v4_retrain_resnet18_kitti_qat.txt \
                    --cal_json_file $USER_EXPERIMENT_DIR/export_qat/cal.json \
                    --target_opset 12 \
                    --gen_ds_config

In [ ]:
print('Exported model:')
print('------------')
!ls -lh $LOCAL_EXPERIMENT_DIR/export_qat

 #### Generate a TensorRT engine using tao deploy

In [ ]:
!tao deploy yolo_v4 gen_trt_engine -m $USER_EXPERIMENT_DIR/export_qat/yolov4_resnet18_epoch_$EPOCH.onnx \
                                   -e $SPECS_DIR/yolo_v4_retrain_resnet18_kitti_qat.txt \
                                   --data_type int8 \
                                   --batch_size 8 \
                                   --min_batch_size 1 \
                                   --opt_batch_size 8 \
                                   --max_batch_size 16 \
                                   --cal_json_file $USER_EXPERIMENT_DIR/export_qat/cal.json \
                                   --engine_file $USER_EXPERIMENT_DIR/export_qat/trt.engine.int8 \
                                   --results_dir $USER_EXPERIMENT_DIR/export_qat

In [ ]:
print('Exported engine:')
print('------------')
!ls -lh $LOCAL_EXPERIMENT_DIR/export_qat/

 ### 12.8. Verify the deployed QAT model <a class="anchor" id="head-12-8"></a>

In [ ]:
!tao deploy yolo_v4 inference -m $USER_EXPERIMENT_DIR/export_qat/trt.engine.int8 \
                              -e $SPECS_DIR/yolo_v4_retrain_resnet18_kitti_qat.txt \
                              -i $DATA_DOWNLOAD_DIR/test_samples \
                              -r $USER_EXPERIMENT_DIR/yolo_infer_images_qat_trt \
                              -t 0.6

In [ ]:
# Visualizing the sample images.
OUTPUT_PATH = 'yolo_infer_images_qat_trt/images_annotated' # relative path from $USER_EXPERIMENT_DIR.
COLS = 3 # number of columns in the visualizer grid.
IMAGES = 9 # number of images to visualize.

visualize_images(OUTPUT_PATH, num_cols=COLS, num_images=IMAGES)